In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import re
import requests
from bs4 import BeautifulSoup
import time
import random

In [ ]:
cookies = {
    'JSESSIONID': 'D12AD85D6F706A7D3E03FC9C6C81035D',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0',
    'Accept': 'application/xml, text/xml, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Origin': 'http://www.iachina.cn',
    'Connection': 'keep-alive',
    'Referer': 'http://www.iachina.cn/col/col41/?uid=1508&pageNum=1',
}

params = (
    ('startrecord', '1'),
    ('endrecord', '107'),
    ('perpage', '40'),
)

data = {
  'col': '1',
  'appid': '1',
  'webid': '1',
  'path': '/',
  'columnid': '41',
  'sourceContentType': '8',
  'unitid': '1508',
  'webname': '\u4E2D\u56FD\u4FDD\u9669\u884C\u4E1A\u534F\u4F1A',
  'permissiontype': '0'
}

response = requests.post('http://www.iachina.cn/module/web/jpage/dataproxy.jsp', headers=headers, params=params, cookies=cookies, data=data)

links=re.findall('<a href="(.*?)">',response.text)
titles=re.findall('html">(.*?)</a>',response.text)
print(len(links))
for link,title in zip(links,titles):
    print(title,'http://www.iachina.cn/'+link)
    with open('web_data.csv','a',encoding='utf-8-sig') as csv:
        csv.write(f'{title},http://www.iachina.cn{link}\n')


In [3]:
web_url = pd.read_csv('web_data.csv',names=['name','url'])
web_url['url'] = web_url['url'].astype(str)
web_url['name'] = web_url['name'].astype(str)
#web_url['url'] = web_url['url'].str.replace('+/','')
web_url.head(100)
name_prefix = web_url['name'].str.findall(r'.*(?=保险.*)')
name_prefix = name_prefix.apply(lambda x:x[0]).to_list()

In [16]:
df = pd.DataFrame(columns = ['Class','TotalEarning','date_time'])
websites = web_url['url'].to_list()
pre_class = None

#try:
for i in range(len(websites)):
    url = websites[i]
    r = requests.get(url)
    print(url)
    soup = BeautifulSoup(r.content,'html.parser')
    table = soup.find('table')
    for row in table.tbody.find_all('tr')[1:]:
        columns = row.find_all('td')
        if (columns[0].text.strip() != ''):
            class_ = columns[0].text.strip()
            earning = columns[1].text.strip()
            date = name_prefix[i]
            if pre_class is not None and '、' in class_:
                if re.match(r'.*_.*',pre_class):
                    pre_class = re.sub(r'_.*', '_'+ class_, pre_class)
                    class_ = pre_class 
                else:
                    class_ = "_".join([pre_class,class_])
                    pre_class = class_

            elif pre_class is not None and '（' in class_:
                    class_ = "_".join([pre_class,class_])
            else:
                pre_class = class_
            df = df.append({'Class': class_, 'TotalEarning':earning,'date_time':date},ignore_index=True)
#except:
    
    
df

http://www.iachina.cn/art/2018/4/3/art_617_102275.html


AttributeError: 'NoneType' object has no attribute 'find_all'

In [23]:
url = 'http://www.iachina.cn/art/2018/4/3/art_617_102275.html'
r = requests.get(url)
#print(url)
soup = BeautifulSoup(r.content,'html.parser')
#table = soup.findall('table')
soup.findall('table')
#table.tbody.find_all('tr')[1:]
#print(table)

TypeError: 'NoneType' object is not callable

In [99]:
df = df.drop_duplicates(subset=['Class','TotalEarning','date_time'])

In [100]:
def replace_chinese_format(x):
    x = x.replace('、','.')
    x = x.replace('（','(')
    x = x.replace('）',')_')
    return x
df['Class']= df['Class'].apply(replace_chinese_format)

/home/frank/miniconda3/envs/mypy/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [120]:
data_d = df.pivot(index = 'date_time',columns='Class',values='TotalEarning').sort_index(ascending = False)

In [121]:
data_d.to_csv('insurance_data_by_frank.csv')

In [111]:
l1 = data_d.index.to_list()
l2 = pd.read_csv('industry_data.csv')['info'].str.findall(r'.*(?=保险.*)')
l2 = l2.apply(lambda x: x[0]).to_list()

In [124]:
set(l1).symmetric_difference(set(l2))

set()

In [125]:
pd.read_csv('industry_data.csv').shape

(107, 23)

In [126]:
data_d.shape

(106, 18)